In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt

# Load the stock data
gstock_data = pd.read_csv('tesla-stock-price.csv')

# Set the date as the index
gstock_data['date'] = pd.to_datetime(gstock_data['date'].apply(lambda x: x.split()[0]))
gstock_data.set_index('date', drop=True, inplace=True)

# Select the 'open' and 'close' columns
gstock_data = gstock_data[['open', 'close']]

# Scale the data
Ms = MinMaxScaler()
gstock_data[gstock_data.columns] = Ms.fit_transform(gstock_data)

# Split the data into training and testing sets
training_size = round(len(gstock_data) * 0.8)
train_data = gstock_data[:training_size]
test_data = gstock_data[training_size:]

# Create sequences for training and testing
def create_sequence(dataset):
    sequences = []
    labels = []
    start_idx = 0
    for stop_idx in range(50, len(dataset)):
        sequences.append(dataset.iloc[start_idx:stop_idx])
        labels.append(dataset.iloc[stop_idx])
        start_idx += 1
    return np.array(sequences), np.array(labels)

train_seq, train_label = create_sequence(train_data)
test_seq, test_label = create_sequence(test_data)

# Reshape the sequences for LSTM
train_seq = np.reshape(train_seq, (train_seq.shape[0], train_seq.shape[1], 2))
test_seq = np.reshape(test_seq, (test_seq.shape[0], test_seq.shape[1], 2))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_seq.shape[1], 2)))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

# Train the model
model.fit(train_seq, train_label, epochs=30, validation_data=(test_seq, test_label), verbose=1)

# Make predictions on the test set
test_predicted = model.predict(test_seq)

# Inverse transform the predictions
test_inverse_predicted = Ms.inverse_transform(test_predicted)

# Visualize the results
gs_slic_data = pd.concat([gstock_data.iloc[-202:].copy(), pd.DataFrame(test_inverse_predicted, columns=['open_predicted', 'close_predicted'], index=gstock_data.iloc[-202:].index)], axis=1)

gs_slic_data[['open', 'open_predicted']].plot(figsize=(10, 6))
plt.xticks(rotation=45)
plt.xlabel('Date', size=15)
plt.ylabel('Stock Price', size=15)
plt.title('Actual vs Predicted for open price', size=15)
plt.show()

gs_slic_data[['close', 'close_predicted']].plot(figsize=(10, 6))
plt.xticks(rotation=45)
plt.xlabel('Date', size=15)
plt.ylabel('Stock Price', size=15)
plt.title('Actual vs Predicted for close price', size=15)
plt.show()

2024-08-01 11:15:51.534691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-01 11:15:51.534761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-01 11:15:51.876964: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 11:15:58.391371: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-01 11:15:58.391585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

Epoch 1/30
18/18 [==============================] - 23s 725ms/step - loss: 0.0798 - mean_absolute_error: 0.2096 - val_loss: 0.0047 - val_mean_absolute_error: 0.0567
Epoch 2/30
18/18 [==============================] - 10s 582ms/step - loss: 0.0119 - mean_absolute_error: 0.0849 - val_loss: 0.0086 - val_mean_absolute_error: 0.0729
Epoch 3/30
18/18 [==============================] - 10s 565ms/step - loss: 0.0060 - mean_absolute_error: 0.0629 - val_loss: 0.0064 - val_mean_absolute_error: 0.0642
Epoch 4/30
18/18 [==============================] - 9s 494ms/step - loss: 0.0056 - mean_absolute_error: 0.0590 - val_loss: 0.0050 - val_mean_absolute_error: 0.0587
Epoch 5/30
18/18 [==============================] - 12s 600ms/step - loss: 0.0052 - mean_absolute_error: 0.0557 - val_loss: 0.0047 - val_mean_absolute_error: 0.0579
Epoch 6/30
18/18 [==============================] - 10s 576ms/step - loss: 0.0050 - mean_absolute_error: 0.0542 - val_loss: 0.0043 - val_mean_absolute_error: 0.0560
Epoch 7/30


ValueError: Shape of passed values is (101, 2), indices imply (202, 2)